# Install cuplyr on Google Colab

GPU-accelerated dplyr for R, installed in ~5 minutes.

## Before running:
1. **Runtime → Change runtime type**
2. Set **Runtime type = R**, **Hardware accelerator = T4 GPU**
3. Click **Save**, then **Run all**

## Step 1: Pre-flight check

In [ ]:
# Verify GPU is attached
gpu_check <- system('nvidia-smi -L', intern = TRUE)
if (length(gpu_check) == 0 || grepl('error|fail', gpu_check[1], ignore.case = TRUE)) {
  stop('No GPU detected! Go to Runtime -> Change runtime type -> GPU')
}
cat('GPU:', gpu_check[1], '\n')

# Find CUDA
for (p in c('/usr/local/cuda', '/usr/local/cuda-12.8', '/usr/local/cuda-12.4', '/usr/local/cuda-12')) {
  if (file.exists(file.path(p, 'include', 'cuda.h'))) {
    Sys.setenv(CUDA_HOME = p)
    break
  }
}
cat('CUDA:', Sys.getenv('CUDA_HOME'), '\n')

# Set up C++20 for R
r_makevars <- path.expand('~/.R/Makevars')
dir.create(dirname(r_makevars), showWarnings = FALSE, recursive = TRUE)
writeLines(c('CXX20=g++', 'CXX20STD=-std=gnu++20', 'CXX20FLAGS=-O2 -fPIC'), r_makevars)

## Step 2: Install mamba (package manager for RAPIDS)

In [ ]:
# Install miniforge (provides mamba)
miniforge_dir <- '/opt/miniforge'
mamba <- file.path(miniforge_dir, 'bin', 'mamba')

if (!file.exists(mamba)) {
  cat('Installing Miniforge...\n')
  system('wget -q https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh -O /tmp/miniforge.sh')
  system(sprintf('bash /tmp/miniforge.sh -b -p %s 2>&1', miniforge_dir))
}

stopifnot('Miniforge install failed' = file.exists(mamba))

# Add mamba to PATH so install_cuplyr() can find it
old_path <- Sys.getenv('PATH')
Sys.setenv(PATH = paste(file.path(miniforge_dir, 'bin'), old_path, sep = ':'))
cat('mamba ready\n')

## Step 3: Install cuplyr

This clones the `install` branch of cuplyr, installs RAPIDS via conda, and builds the package. Takes 3-5 minutes on first run.

In [ ]:
# Clone cuplyr install branch from GitHub
repo_dir <- '/content/cuplyr'
repo_branch <- 'install'
if (dir.exists(repo_dir)) unlink(repo_dir, recursive = TRUE)

status <- system2('git', c('clone', '--depth', '1', '-b', repo_branch,
                           'https://github.com/bbtheo/cuplyr.git', repo_dir),
                  stdout = FALSE, stderr = FALSE)
if (status != 0) {
  stop('Git clone failed. Check network connection or branch name.', call. = FALSE)
}

# Source the install functions
source(file.path(repo_dir, 'R', 'check-deps.R'))
source(file.path(repo_dir, 'R', 'install.R'))

# Run installer with Colab-specific settings
Sys.setenv(CUPLYR_ENV = 'colab')
install_cuplyr(
  repo = repo_dir,
  method = 'conda',
  conda_prefix = '/opt/rapids',
  verbose = TRUE
)

## Step 4: Load cuplyr

CRITICAL: We must manually preload the RAPIDS libstdc++ before loading cuplyr to ensure libcudf.so finds the correct GLIBCXX_3.4.31 symbols.

In [ ]:
# Helper to follow symlinks to the real file
resolve_symlink <- function(path) {
  if (!file.exists(path)) return(NULL)
  link <- Sys.readlink(path)
  if (is.na(link) || link == "") {
    return(path)
  }
  if (startsWith(link, '/')) {
    return(resolve_symlink(link))
  }
  resolve_symlink(file.path(dirname(path), link))
}

# Verify RAPIDS libstdc++ exists and has GLIBCXX_3.4.31
cat('Checking RAPIDS libstdc++...\n')
conda_stdc_symlink <- '/opt/rapids/lib/libstdc++.so.6'
conda_stdc <- resolve_symlink(conda_stdc_symlink)

if (is.null(conda_stdc) || !file.exists(conda_stdc)) {
  stop(
    'RAPIDS libstdc++.so.6 not found.\n',
    'Expected at: ', conda_stdc_symlink, '\n',
    'RAPIDS installation may have failed in Step 3.',
    call. = FALSE
  )
}

# Check for GLIBCXX_3.4.31
glibcxx_check <- system2(
  'strings',
  conda_stdc,
  stdout = TRUE,
  stderr = FALSE
)

if (!any(grepl('GLIBCXX_3\\.4\\.31', glibcxx_check))) {
  stop(
    'RAPIDS libstdc++ does not contain GLIBCXX_3.4.31.\n',
    'File: ', conda_stdc, '\n',
    'The RAPIDS environment may be outdated or incomplete.',
    call. = FALSE
  )
}

cat('  ✓ Found:', conda_stdc, '\n')
cat('  ✓ Contains GLIBCXX_3.4.31\n\n')

# Preload RAPIDS libstdc++ to override system version
cat('Preloading RAPIDS libstdc++...\n')
preload_result <- tryCatch({
  # Force the RAPIDS libstdc++ to be loaded first using dyn.load
  # This ensures that when cuplyr.so loads libcudf.so, the correct
  # libstdc++ is already in the process and will be used
  dyn.load(conda_stdc, local = FALSE, now = TRUE)
  cat('  ✓ Preloaded:', conda_stdc, '\n\n')
  TRUE
}, error = function(e) {
  cat('  ✗ Failed to preload libstdc++\n')
  cat('  Error:', conditionMessage(e), '\n\n')
  FALSE
})

if (!preload_result) {
  stop('Failed to preload RAPIDS libstdc++. Cannot proceed.', call. = FALSE)
}

# Now load cuplyr - libcudf should find the preloaded libstdc++
cat('Loading cuplyr package...\n')
load_error <- tryCatch({
  library(cuplyr)
  cat('  ✓ cuplyr loaded successfully!\n\n')
  NULL
}, error = function(e) {
  cat('  ✗ Failed to load cuplyr\n')
  cat('  Error:', conditionMessage(e), '\n\n')
  e
})

if (!is.null(load_error)) {
  cat('=== DIAGNOSTICS ===\n\n')
  
  cat('1. Check what cuplyr.so depends on:\n')
  cat('   ldd /usr/local/lib/R/site-library/cuplyr/libs/cuplyr.so | grep -E "libstdc\\+\\+|libcudf|libcuda"\n\n')
  
  cat('2. Check what libcudf.so depends on:\n')
  cat('   ldd /opt/rapids/lib/libcudf.so | grep libstdc\\+\\+\n\n')
  
  cat('3. Verify GLIBCXX symbols in system libstdc++:\n')
  cat('   strings /usr/lib/x86_64-linux-gnu/libstdc++.so.6 | grep GLIBCXX_3.4\n\n')
  
  cat('4. Check currently loaded libraries in this R session:\n')
  cat('   getLoadedDLLs()\n\n')
  
  stop('cuplyr failed to load. See diagnostics above.', call. = FALSE)
}

# Verify installation
source(file.path(repo_dir, 'R', 'verify.R'))
verify_installation()

## Step 5: Try cuplyr

GPU-accelerated dplyr — same syntax, runs on GPU.

In [ ]:
# Filter, mutate, select, arrange
tbl_gpu(mtcars) |>
  filter(mpg > 20) |>
  mutate(kpl = mpg * 0.425) |>
  select(mpg, kpl, cyl, hp) |>
  arrange(desc(mpg)) |>
  collect()

In [ ]:
# Group by and summarise
tbl_gpu(mtcars) |>
  group_by(cyl) |>
  summarise(
    count = n(),
    avg_mpg = mean(mpg),
    avg_hp = mean(hp)
  ) |>
  arrange(cyl) |>
  collect()

In [ ]:
# Lazy evaluation with query optimization
tbl_gpu(mtcars, lazy = TRUE) |>
  filter(mpg > 15) |>
  mutate(power_weight = hp / wt) |>
  group_by(cyl) |>
  summarise(avg_pw = mean(power_weight)) |>
  arrange(desc(avg_pw)) |>
  collect()

## Troubleshooting

Run `diagnostics()` if GPU is not detected — paste the output in a [GitHub issue](https://github.com/bbtheo/cuplyr/issues).

In [ ]:
# Full diagnostics — paste this output in bug reports
source(file.path(repo_dir, 'R', 'diagnostics.R'))
diagnostics(redact = FALSE)

# Dependency check
check_deps()